In [382]:
import pandas as pd
import os, sys
from keras.models import Sequential
from keras.layers import *
import keras
import tensorflow as tf
import numpy as np
import math
import random
from tensorflow.keras import regularizers , initializers
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import json
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [383]:
files = os.listdir('RBNS_training')+os.listdir('RBNS_testing')
FILES = [file for file in files if  file.startswith("RBP1_")]
FILES

['RBP1_1300nM.seq',
 'RBP1_20nM.seq',
 'RBP1_320nM.seq',
 'RBP1_5nM.seq',
 'RBP1_80nM.seq',
 'RBP1_input.seq']

In [384]:
file_num = 1 #RBPi 1<i<31
names = ['input','1300nM']
labels = [0,1]

In [385]:
def generate_data(i,names):
    concentrations = list()
    for name in names: 
        if i > 16:
            addres =  'RBNS_testing/RBP{}_{}.seq'.format(file_num ,name) 
        else :
            addres =  'RBNS_training/RBP{}_{}.seq'.format(file_num ,name)
        print("reading " + addres)
        df = pd.read_csv(addres, sep="\t",usecols=[0],names=['sequence'])
        df = df[0:100000]
        print("data shape with N = " + str(df.shape[0]))
        df = [seq for seq in df['sequence'] if all(ch != 'N' for ch in seq)] 
        df = pd.DataFrame(data=df,columns=['sequence'])
        print("data shape without N = "+ str(df.shape[0]))
        concentrations.append(df)
    return concentrations 

def label_data(RBP,labels):
    for j  in range(0,len(RBP)): 
        RBP[j]['label'] = labels[j]
    return RBP

def merge_data(RBP):
    data = RBP[0]
    for j  in range(1,len(RBP)): 
        data = data.append(RBP[j], ignore_index=True)
    return data

In [386]:
import numpy as np
# from string import maketrans    # in python free use str.maketrans isntead
def oneHot(string):
	trantab=str.maketrans('ACGTU','01233')
	string=str(string)
	data=[int(x) for x in list(string.translate(trantab))]
	return np.eye(4)[data]

In [387]:
RBP = generate_data(i,names)
RBP = label_data(RBP,labels)
RBP = merge_data(RBP)
RBP = RBP.sample(frac=1)
RBP = shuffle(RBP)
RBP

reading RBNS_training/RBP1_input.seq
data shape with N = 100000
data shape without N = 99929
reading RBNS_training/RBP1_1300nM.seq
data shape with N = 100000
data shape without N = 99930


,sequence,label
63803,AGTCACCACTTGGATAAAGA,0
134575,CAACATGACAAAAAGAGGCA,1
4735,AGTAAAGTACACATCCTCGG,0
124061,GTAAGGCACGAGCGTTAAAG,1
108177,AGCTATCTCGTCTGGCTGGG,1
...,...,...
111275,GTGTAACTAAAGAGTGTGTG,1
119023,CACATAAGAAGCAGCGGCGG,1
34269,CGCTCTTGCGGGGGAAAGCT,0
145749,TAATCGAACGGTCATTTATC,1


In [388]:
X = RBP["sequence"]
Y = RBP["label"].values
X = np.array(list(map(oneHot,X)))
print(X.shape)
print(Y.shape)

(199859, 20, 4)
(199859,)


In [389]:
def logsampler(a,b):
        x=np.random.uniform(low=0,high=1)
        y=10**((math.log10(b)-math.log10(a))*x + math.log10(a))
        return y
    
def sqrtsampler(a,b):
        x=np.random.uniform(low=0,high=1)
        y=(b-a)*math.sqrt(x)+a
        return y

In [390]:
def DeepBind(motif_length,number_of_motifs,neuType,learning_rate,momentum_rate,
              dropout_prob,sigmaConv,sigmaNeu,beta1,beta2):
        model=Sequential()
        model.add(Conv1D(filters=number_of_motifs, kernel_size=motif_length, strides=1,
                         activation='relu', input_shape=(20,4), use_bias=True, padding = 'same',
                        kernel_regularizer=regularizers.l2(beta1), bias_regularizer=regularizers.l2(beta1),
                        kernel_initializer=initializers.RandomNormal(stddev=sigmaConv),
                        bias_initializer=initializers.Zeros()))
        model.add(MaxPooling1D(pool_size=3))
        model.add(Dropout(1- dropout_prob))
        model.add(Flatten())
        if neuType == '32' :
            model.add(Dense(units=32,kernel_regularizer=regularizers.l2(beta2), bias_regularizer=regularizers.l2(beta2),
                            kernel_initializer=initializers.RandomNormal(stddev=sigmaNeu),
                            bias_initializer=initializers.Zeros()))
            model.add(Dropout(1 - dropout_prob))
        model.add(Dense(1, activation='sigmoid'))

        opt = keras.optimizers.SGD(lr=learning_rate,momentum = momentum_rate,decay=1e-6)
        model.compile(loss='binary_crossentropy',
                      optimizer=opt,
                      metrics=[tf.keras.metrics.AUC()])   
        return model

In [391]:
def Yaron():
    model=Sequential()
    model.add(Conv1D(filters=128, kernel_size=5, strides=1, activation='relu', input_shape=(20,4), use_bias=False))
    model.add(MaxPooling1D(pool_size=5,strides=1))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

        # initiate RMSprop optimizer
    opt = keras.optimizers.Adam(lr=0.0005)

    # Let's train the model using RMSprop
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    return model

In [392]:
model = Yaron()
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 16, 128)           2560      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 12, 128)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1536)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                98368     
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_11 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                

In [394]:
x_train, x_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42)

In [395]:
history = model.fit(x_train,y_train,
          batch_size=32,
          epochs=3,
          validation_data = (x_test, y_test),
          verbose=2)
# Generate generalization metrics

Epoch 1/3
4185/4185 - 9s - loss: 0.6734 - accuracy: 0.5724 - val_loss: 0.6668 - val_accuracy: 0.5851
Epoch 2/3
4185/4185 - 10s - loss: 0.6669 - accuracy: 0.5839 - val_loss: 0.6650 - val_accuracy: 0.5859
Epoch 3/3
4185/4185 - 11s - loss: 0.6644 - accuracy: 0.5890 - val_loss: 0.6662 - val_accuracy: 0.5868


In [14]:
# Define the K-fold Cross Validator
num_folds = 3
kfold = KFold(n_splits=num_folds, shuffle=False)
for train, valid in kfold.split(X,Y):
    print('valid = ' + str(valid))
    print('train = ' + str(train))

valid = [    0     1     2 ... 66617 66618 66619]
train = [ 66620  66621  66622 ... 199856 199857 199858]
valid = [ 66620  66621  66622 ... 133237 133238 133239]
train = [     0      1      2 ... 199856 199857 199858]
valid = [133240 133241 133242 ... 199856 199857 199858]
train = [     0      1      2 ... 133237 133238 133239]


In [15]:
BEST_AUC = 0
for calibration in range (1) :
    motif_length = random.choice([5,6])
    number_of_motifs = random.choice([64,128])
    neuType = random.choice(['none','32'])
    learning_rate = logsampler(0.0005,0.05)
    momentum_rate=sqrtsampler(0.95,0.99)
    batch_size = 128
    epochs = 10
    dropout_prob =random.choice([0.5,0.75,1.0])    
    sigmaConv=logsampler(10**-7,10**-3)   
    sigmaNeu=logsampler(10**-5,10**-2) 
    beta1=logsampler(10**-15,10**-3)
    beta2=logsampler(10**-10,10**-3) 
    model_acc = []
    # K-fold Cross Validation model evaluation
    fold_no = 1
    for train, valid in kfold.split(X,Y):
        #model = Yaron()
              # Fit data to model
        history = model.fit(X[train], Y[train],
                  batch_size=batch_size,
                  validation_data = (X[valid], Y[valid]),
                  epochs=epochs,
                  verbose=2)
          # Generate generalization metrics
        scores = model.evaluate(X[valid], Y[valid], verbose=0)
        print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}, AUC of {scores[1]*100}%')
        model_acc.append(scores[1])
        # Increase fold number
        fold_no = fold_no + 1
    AUC = np.mean(model_acc)
    print(f'mean AUC = {AUC}')
    if AUC > BEST_AUC:
        best_AUC=AUC
        best_motif_length = motif_length
        best_number_of_motifs = number_of_motifs
        best_neuType = neuType
        best_Learning_rate = learning_rate
        best_momentum_rate = momentum_rate
        best_batch_size = batch_size
        best_epochs = epochs
        best_dropout_prob = dropout_prob
        best_sigmaConv = sigmaConv 
        best_sigmaNeu = sigmaNeu
        best_beta1 = beta1
        best_beta2 = beta2

Epoch 1/10
1041/1041 - 5s - loss: 0.6613 - accuracy: 0.5928 - val_loss: 0.6618 - val_accuracy: 0.5935
Epoch 2/10
1041/1041 - 6s - loss: 0.6601 - accuracy: 0.5936 - val_loss: 0.6613 - val_accuracy: 0.5935
Epoch 3/10
1041/1041 - 5s - loss: 0.6585 - accuracy: 0.5975 - val_loss: 0.6625 - val_accuracy: 0.5923
Epoch 4/10
1041/1041 - 5s - loss: 0.6572 - accuracy: 0.6001 - val_loss: 0.6625 - val_accuracy: 0.5905
Epoch 5/10
1041/1041 - 6s - loss: 0.6553 - accuracy: 0.6019 - val_loss: 0.6635 - val_accuracy: 0.5911
Epoch 6/10
1041/1041 - 5s - loss: 0.6531 - accuracy: 0.6063 - val_loss: 0.6654 - val_accuracy: 0.5886
Epoch 7/10
1041/1041 - 5s - loss: 0.6507 - accuracy: 0.6082 - val_loss: 0.6657 - val_accuracy: 0.5867
Epoch 8/10
1041/1041 - 6s - loss: 0.6478 - accuracy: 0.6125 - val_loss: 0.6687 - val_accuracy: 0.5848
Epoch 9/10
1041/1041 - 5s - loss: 0.6448 - accuracy: 0.6173 - val_loss: 0.6703 - val_accuracy: 0.5840
Epoch 10/10
1041/1041 - 5s - loss: 0.6416 - accuracy: 0.6212 - val_loss: 0.6719 - 

In [16]:
best_hyperparameters = {'Best AUC':best_AUC,'Best Motif length':best_motif_length,'Best num Motif':best_number_of_motifs,'Best NeuType':best_neuType,'Best Learning Rate':best_Learning_rate,
                        'Best Momentum rate':best_momentum_rate,'Best SigmaConv':best_sigmaConv,'Best Dropout':best_dropout_prob,
                        'Best sigmaNeu':best_sigmaNeu,'Best Beta 1':best_beta1, 'Best Beta 2':best_beta2}

In [17]:
motif_length = best_hyperparameters['Best Motif length']
number_of_motifs = best_hyperparameters['Best num Motif']
neuType = best_hyperparameters['Best NeuType']
neuType = best_hyperparameters['Best NeuType']
learning_rate = best_hyperparameters['Best Learning Rate']
momentum_rate = best_hyperparameters['Best Momentum rate']
dropout_prob = best_hyperparameters['Best Dropout']
sigmaConv = best_hyperparameters['Best SigmaConv']
sigmaNeu = best_hyperparameters['Best sigmaNeu']
beta1 = best_hyperparameters['Best Beta 1']
beta2 = best_hyperparameters['Best Beta 2']

In [22]:
best_model = DeepBind(motif_length,number_of_motifs,neuType,learning_rate,momentum_rate,
              dropout_prob,sigmaConv,sigmaNeu,beta1,beta2)
best_model.summary()
      # Fit data to model
history = best_model.fit(X,Y,
          batch_size = batch_size,
          epochs=epochs,
          verbose=2)
  # Generate generalization metrics


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 20, 64)            1600      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 6, 64)             0         
_________________________________________________________________
dropout (Dropout)            (None, 6, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 385       
Total params: 1,985
Trainable params: 1,985
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1562/1562 - 4s - loss: 0.6877 - auc: 0.5739
Epoch 2/10
1562/1562 - 2s - loss: 0.6799 - auc: 0.5930
Epoch 3/

In [23]:
best_model.save("Results/RBP{}/best_model".format(file_num))
best_model.save("Results/RBP{}/Best".format(file_num))
model.save_weights("Results/RBP{}/Weights".format(file_num))

INFO:tensorflow:Assets written to: Results/RBP1/best_model\assets
INFO:tensorflow:Assets written to: Results/RBP1/Best\assets


In [24]:
for hyper in best_hyperparameters:
    print("{} = {}".format(hyper, best_hyperparameters[hyper]))
a_file = open("Results/RBP{}/Best.json".format(i), "w")
json.dump(best_hyperparameters, a_file)
a_file.close()

Best AUC = 0.5995677908261617
Best Motif length = 6
Best num Motif = 64
Best NeuType = none
Best Learning Rate = 0.0005943017622589472
Best Momentum rate = 0.9663550833728601
Best SigmaConv = 3.6978806313760743e-07
Best Dropout = 0.5
Best sigmaNeu = 0.006643408312224844
Best Beta 1 = 4.758771682176259e-13
Best Beta 2 = 1.88545699397436e-05


In [480]:
x_result= pd.read_csv('RNCMPT_training/seq.txt', sep="\t", header=None)
y_result = pd.read_csv('RNCMPT_training/RBP{}.txt'.format(file_num), sep="\t", header=None)
y_result = np.asarray(y_result[0])
y_result = y_result
x_result = x_result
len(x_result)

241357

In [481]:
x_result =list(map(oneHot,x_result[0]))

In [482]:
lengths = [len(x_result[i])-20+1 for i in range(len(x_result))]


In [483]:
length = iter(lengths)
padded = [x[i:i+20] for x in x_result for i in range(next(length))]
padded = np.asarray(padded)
len(padded)


4117065

In [484]:
y_predict = model.predict(padded) 
print(y_predict.shape)
splits = [0] * (len(lengths)-1)
splits[0] = lengths[0] 
for i in range(1,len(lengths)-1):
    splits[i] += (lengths[i] + splits[i-1])
y_predict = np.asarray(np.split(y_predict,splits))



(4117065, 1)


In [501]:
y_hat = [np.max(result) for result in y_predict]
len(y_hat)

241357

In [502]:
y_hat = np.asarray(y_hat)

In [503]:
df = pd.DataFrame(y_hat)
df.to_csv("Results/RBP{}/Prediction.txt".format(file_num),sep="\t", index=False,header=False)

In [504]:
from scipy.stats.stats import pearsonr 
print(f'pearson correlation = {pearsonr(y_result,y_hat)}')

pearson correlation = (0.4461803006708791, 0.0)
